# EXTRACT
# Scraping Women t-shirts from ASOS API (2)


(2) Extract more products similar to tshirt category

### 0. Import modules

In [3]:
import requests
import time
import pandas as pd
import pickle

In [4]:
from logs.PASSES import RAPID ## my TOKEN

In [5]:
len(RAPID)

50

## RapidAPI - Asos

RapidAPI is an API Marketplace for developers to find, connect, and manage their API connections. 
https://rapidapi.com/apidojo/api/zappos1

In [7]:
headers = {
    "X-RapidAPI-Key": RAPID,
    "X-RapidAPI-Host": "asos2.p.rapidapi.com"
}

### Get countries available to get ASOS data from

Extract the relevant info for **SPAIN**

In [8]:
url = "https://asos2.p.rapidapi.com/countries/list"

response = requests.get(url, headers=headers)

countries_res=response.json()

In [9]:
ES_index = [index for index, countries in enumerate(countries_res) if countries.get("country") == "ES"]
nombre_pais=countries_res[ES_index[0]]["name"]
pais=countries_res[ES_index[0]]["country"]
idioma=countries_res[ES_index[0]]["languages"][0]["language"]
moneda=countries_res[ES_index[0]]["currencies"][0]["currency"]

display(nombre_pais, pais, idioma, moneda)

'Spain'

'ES'

'es-ES'

'EUR'

### Get data directly from `category_id`

In [15]:
url = "https://asos2.p.rapidapi.com/categories/list"

querystring = {"country":"pais", "lang":"idioma"}

response = requests.get(url, headers=headers, params=querystring)

res=response.json()

In [16]:
res["navigation"][0] #men
res["navigation"][1] #women
resWomen=res["navigation"][1]

In [ ]:
# resWomen

### Create a list containing the type of items and their `category_id`

In [18]:
category_list=[]
for RESW in resWomen["children"]:
   # display(RESW["children"])
    for i in range(len(RESW["children"])):
        RESW2=RESW["children"][i]["children"]
        for j in RESW2:
            RESW3=j["children"]
            for n in RESW3:
                #display(n["link"]["categoryId"])
                #display(n["link"]["webUrl"])
                category_list.append(n["link"]["webUrl"])

In [19]:
category_list2=[CAT.split("cid=") for CAT in category_list]
category_list2=[CAT for CAT in category_list2 if len(CAT)==2]

In [1]:
#category_list2

### `category_id` of Woman T-shirts is # 20581

In [ ]:
#find_category="t-shirt"
#find_category="top"
find_category="shirt"


category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)
        
category_listx

### "shirts" `category_id` 

In [ ]:
#find_category="t-shirt"
#find_category="top"
find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)
        
category_listx

In [78]:
categories=[VALS[1] for VALS in category_listx]
categories=list(set(categories))
categories_shirt=categories

['15200', '20581', '13053', '27423', '11321', '20580']

### "tops" `category_id` 

In [130]:
#find_category="t-shirt"
find_category="top"
#find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)

category_listx=[VALS for VALS in category_listx if "brands" not in  VALS[0]]
category_listx=[VALS[1] for VALS in category_listx]
categories=[VALS.split("&")[0] for VALS in category_listx]
categories_top=categories
categories_top

['20581', '4167', '4167', '20581', '4169', '27167', '9581', '9581', '27421']

In [117]:
find_category="t-shirt"
#find_category="top"
#find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)
category_listx     

[['https://www.asos.com/us/women/sale/tops/t-shirts-tank-tops/cat/?', '20581'],
 ['https://www.asos.com/us/women/sale/tops/t-shirts-tank-tops/cat/?', '20581']]

In [99]:
categories=[VALS[1] for VALS in category_listx]
categories=list(set(categories))
categories=[VALS.split("&")[0] for VALS in categories]
categories_top=categories

### dresses `category_id` 

In [ ]:
#find_category="t-shirt"
find_category="dresses"
#find_category="shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)

category_listx=[VALS for VALS in category_listx if "brands" not in  VALS[0]]
category_listx=[VALS[1] for VALS in category_listx]
categories=[VALS.split("&")[0] for VALS in category_listx]
categories_dress=categories


### ITERATE THE SEARCH TO GET ALL THE PRODUCTS 

Change the category id's to find different types of products

In [143]:
url = "https://asos2.p.rapidapi.com/products/v2/list"
headers = {
    "X-RapidAPI-Key": RAPID,
    "X-RapidAPI-Host": "asos2.p.rapidapi.com"
}

# categories_dress
# "20581" #only t-shirt
# categories_top
# categories_shirt


category_id = categories_dress # "20581"
limit = 48
offset = 0
results = []

while True:
    querystring = {
        "categoryId": categories_dress, #"20581",
        "lang":{idioma},"currency":{moneda},"store":{pais},"country":{pais},
        "limit": str(limit),
        "offset": str(offset),
        "sort": "freshness"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        results.extend(data['products'])  # Append the products from the current page to results
        offset += limit  # Increment the offset to get the next page
        if len(data['products']) < limit:
            break  # Break the loop if the current page has fewer than the limit
    else:
        print(f"Error: {response.status_code}")
        break  # Exit the loop on error



In [148]:
thedf=[[row["brandName"], row["name"],  row["price"]["current"]["text"], row["colour"]] for row in results]
tets_df=pd.DataFrame(thedf,columns=["brand", "description", "price", "colour"])
tets_df

,brand,description,price,colour
0,Threadbare,Vestido blanco y negro 2 en 1 estilo jersey co...,"29,99 €",Negro/blanco
1,In Wear,Vestido midi marrón con mangas abullonadas de ...,"108,99 €",Negro
2,Gilli,Vestido midi color oliva con escote corazón de...,"33,99 €",Oliva
3,QED London,Vestido corto a cuadros vichy azules de corte ...,"34,99 €",Azul/blanco
4,Vero Moda,Vestido corto verde estilo jersey de manga lar...,"42,99 €",Dark forest
...,...,...,...,...
7433,Liquorish,Vestido midi cruzado con bajo tipo pañuelo y e...,"34,95 €",Multicolor
7434,& Other Stories,Minivestido negro con manga abullonada de & Ot...,"35,05 €",NEGRO
7435,Selected,Vestido de cambray a rayas con detalle de punt...,"23,00 €",Blanco
7436,ASOS Tall,Vestido estilo peto denim en negro desgastado ...,"18,49 €",WASHED BLACK


In [144]:
#asos_womendress_data=results #only t-shirts
## asos_womentshirt_data=results #only t-shirts
## asos_womenshirt2_data=results # all "shirts"
#asos_womentop_data=results # all "shirts"

In [145]:
len(asos_womendress_data)

7438

### Save the data using pickle

Comment the lines to avoid overwriting

In [146]:
### Save the object to a file using pickle
#with open('asos_womendress_data.pkl', 'wb') as file:
#    pickle.dump(asos_womendress_data, file)

### Save the object to a file using pickle
#with open('asos_womentop_data.pkl', 'wb') as file:
#    pickle.dump(asos_womentop_data, file)

### Save the object to a file using pickle
#with open('asos_womenshirt2_data.pkl', 'wb') as file:
#    pickle.dump(asos_womenshirt2_data, file)
    
### Save the object to a file using pickle
#with open('asos_womentshirt_data.pkl', 'wb') as file:
#    pickle.dump(asos_womentshirt_data, file)

# Now the same for men

### Create a list containing the type of items and their `category_id`

In [ ]:
category_list=[]
for RESW in resMen["children"]:
   # display(RESW["children"])
    for i in range(len(RESW["children"])):
        RESW2=RESW["children"][i]["children"]
        for j in RESW2:
            RESW3=j["children"]
            for n in RESW3:
                #display(n["link"]["categoryId"])
                #display(n["link"]["webUrl"])
                category_list.append(n["link"]["webUrl"])

In [ ]:
category_list2=[CAT.split("cid=") for CAT in category_list]
category_list2=[CAT for CAT in category_list2 if len(CAT)==2]

### `category_id` of Men T-shirts

In [ ]:
find_category="t-shirt"

category_listx=[]
for CAT in category_list2:
    if(find_category in CAT[0]):
        category_listx.append(CAT)
        
categories=[VALS[1] for VALS in category_listx]
categories=list(set(categories))
categories_Tshirt=categories

### ITERATE THE SEARCH TO GET ALL THE PRODUCTS 

Change the category id's to find different types of products

In [ ]:
url = "https://asos2.p.rapidapi.com/products/v2/list"
headers = {
    "X-RapidAPI-Key": RAPID,
    "X-RapidAPI-Host": "asos2.p.rapidapi.com"
}

# categories_Tshirt


category_id = categories_Tshirt #
limit = 48
offset = 0
results = []

while True:
    querystring = {
        "categoryId": "27449", #"20581",
        "lang":{idioma},"currency":{moneda},"store":{pais},"country":{pais},
        "limit": str(limit),
        "offset": str(offset),
        "sort": "freshness"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        results.extend(data['products'])  # Append the products from the current page to results
        offset += limit  # Increment the offset to get the next page
        if len(data['products']) < limit:
            break  # Break the loop if the current page has fewer than the limit
    else:
        print(f"Error: {response.status_code}")
        break  # Exit the loop on error



In [ ]:
thedf=[[row["brandName"], row["name"],  row["price"]["current"]["text"], row["colour"]] for row in results]
tets_df=pd.DataFrame(thedf,columns=["brand", "description", "price", "colour"])
tets_df

In [ ]:
#asos_mentshirt_data1=results #only t-shirts
#asos_mentshirt_data2=results #only t-shirts

### Save the data using pickle

Comment the lines to avoid overwriting

In [ ]:
### Save the object to a file using pickle
#with open('../data/raw/asos_mentshirt_data1.pkl', 'wb') as file:
#   pickle.dump(asos_mentshirt_data1, file)

### Save the object to a file using pickle
#with open('../data/raw/asos_mentshirt_data2.pkl', 'wb') as file:
#    pickle.dump(asos_mentshirt_data2, file)